In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from PIL import Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import scipy.ndimage
import scipy.misc
%matplotlib inline
import h5py
import gc

In [ ]:
dataset_location = 'datasets/svhn_raw/'

In [ ]:
# The DigitStructFile is just a wrapper around the h5py data.  It basically references 
#    inf:              The input h5 matlab file
#    digitStructName   The h5 ref to all the file names
#    digitStructBbox   The h5 ref to all struc data
class DigitStructFile:
    def __init__(self, inf):
        self.inf = h5py.File(inf, 'r')
        self.digitStructName = self.inf['digitStruct']['name']
        self.digitStructBbox = self.inf['digitStruct']['bbox']

# getName returns the 'name' string for for the n(th) digitStruct. 
    def getName(self,n):
        return ''.join([chr(c[0]) for c in self.inf[self.digitStructName[n][0]].value])

# bboxHelper handles the coding difference when there is exactly one bbox or an array of bbox. 
    def bboxHelper(self,attr):
        if (len(attr) > 1):
            attr = [self.inf[attr.value[j].item()].value[0][0] for j in range(len(attr))]
        else:
            attr = [attr.value[0][0]]
        return attr

# getBbox returns a dict of data for the n(th) bbox. 
    def getBbox(self,n):
        bbox = {}
        bb = self.digitStructBbox[n].item()
        bbox['height'] = self.bboxHelper(self.inf[bb]["height"])
        bbox['label'] = self.bboxHelper(self.inf[bb]["label"])
        bbox['left'] = self.bboxHelper(self.inf[bb]["left"])
        bbox['top'] = self.bboxHelper(self.inf[bb]["top"])
        bbox['width'] = self.bboxHelper(self.inf[bb]["width"])
        return bbox

    def getDigitStructure(self,n):
        s = self.getBbox(n)
        s['name']=self.getName(n)
        return s

# getAllDigitStructure returns all the digitStruct from the input file.     
    def getAllDigitStructure(self):
        return [self.getDigitStructure(i) for i in range(len(self.digitStructName))]

# Return a restructured version of the dataset (one structure by boxed digit).
#
#   Return a list of such dicts :
#      'filename' : filename of the samples
#      'boxes' : list of such dicts (one by digit) :
#          'label' : 1 to 9 corresponding digits. 10 for digit '0' in image.
#          'left', 'top' : position of bounding box
#          'width', 'height' : dimension of bounding box
#
# Note: We may turn this to a generator, if memory issues arise.
    def getAllDigitStructure_ByDigit(self):
        pictDat = self.getAllDigitStructure()
        result = []
        structCnt = 1
        for i in range(len(pictDat)):
            item = { 'filename' : pictDat[i]["name"] }
            figures = []
            for j in range(len(pictDat[i]['height'])):
               figure = {}
               figure['height'] = pictDat[i]['height'][j]
               figure['label']  = pictDat[i]['label'][j]
               figure['left']   = pictDat[i]['left'][j]
               figure['top']    = pictDat[i]['top'][j]
               figure['width']  = pictDat[i]['width'][j]
               figures.append(figure)
            structCnt = structCnt + 1
            item['boxes'] = figures
            result.append(item)
        return result

In [ ]:
def load_image(file_name, folder_name) :
    image_name = dataset_location + folder_name + '/' + file_name
    img = Image.open(image_name).convert('L')
    img.load()
    data = np.asarray(img, dtype="int32")
    return data

In [ ]:
def plot_img(image):
    plt.imshow(image)
    plt.show()

In [ ]:
# fin = os.path.join(dataset_location + 'test', 'digitStruct.mat')
# dsf = DigitStructFile(fin)
# test_data = dsf.getAllDigitStructure_ByDigit()

In [ ]:
# fin = os.path.join(dataset_location + 'train', 'digitStruct.mat')
# dsf = DigitStructFile(fin)
# train_data = dsf.getAllDigitStructure_ByDigit()

In [ ]:
# fin = os.path.join(dataset_location + 'extra', 'digitStruct.mat')
# dsf = DigitStructFile(fin)
# extra_data = dsf.getAllDigitStructure_ByDigit()

In [ ]:
pickle_file = 'datasets/pickles/SVHN_new_data_struct.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    test_data = save['test_data']
    train_data = save['train_data']
    extra_data = save['extra_data']
    del save
print(len(train_data))
print(len(test_data))
print(len(extra_data))

In [ ]:
test_images = np.ndarray([len(test_data), 32, 96, 1], dtype='int32')
test_labels = np.ndarray([len(test_data), 6], dtype='int32')
folder_name = 'test'

l = len(test_data)

for i in range(l) :
    image = test_data[i]
    im = load_image(image['filename'], folder_name)
    # Resizing Image to be of 32x96x3 dimensions
    im = scipy.misc.imresize(im, (32, 96))
    
    # Finding No of Digits in given image
    no_of_digits = len(image['boxes'])
    # If no of digits is > 5
    # Leave this example out, as we don't need it
    # Continue to next Iteration
    if no_of_digits > 5:
        print('more then 5 digits', image['boxes'])
        continue

    # Numpy Array 'dig' will contain digits in the format :
    # [No_of_digits, _, _, _, _, _]
    dig = np.array([])
    dig = np.append(dig, no_of_digits)
    
    for j in range(no_of_digits) :
        digit = image['boxes'][j]['label']
        dig = np.append(dig, digit)

    # Appending '0' to represent Empty Space
    zeros = 5 - no_of_digits
    for z in range(zeros) :
        dig = np.append(dig, 0)
        

    im = im[:,:,np.newaxis]
    test_images[i,:,:,:] = im[:,:,:]
    test_labels[i,:] = dig
    
    if i % 1500 == 0:
        print((i/l)*100)
        

In [ ]:
l = len(train_data)
more_then_5_counter = 0

train_images = np.ndarray([l, 32, 96, 1], dtype='int32')
train_labels = np.ndarray([l, 6], dtype='int32')
folder_name = 'train'


for i in range(l) :
    image = train_data[i]
    im = load_image(image['filename'], folder_name)
    im = scipy.misc.imresize(im, (32, 96))
    
    no_of_digits = len(image['boxes'])
    if no_of_digits > 5:
        more_then_5_counter += 1
        continue

    dig = np.array([])
    dig = np.append(dig, no_of_digits)

    for j in range(no_of_digits) :
        digit = image['boxes'][j]['label']
        dig = np.append(dig, digit)

    zeros = 5 - no_of_digits
    for z in range(zeros) :
        dig = np.append(dig, 0)
        
    im = im[:,:,np.newaxis]
    train_images[i,:,:,:] = im[:,:,:]
    train_labels[i,:] = dig
    
    
    if i % 5000 == 0:
        print('progress : ', (i/l)*100, '%')
        
print('Cases Containing More then 5 Digits : ', more_then_5_counter)

In [ ]:
l = len(extra_data)
more_then_5_counter = 0

extra_images = np.ndarray([l, 32, 96, 1], dtype='int32')
extra_labels = np.ndarray([l, 6], dtype='int32')
folder_name = 'extra'

for i in range(l) :
    image = extra_data[i]
    im = load_image(image['filename'], folder_name)
    im = scipy.misc.imresize(im, (32, 96))
    
    no_of_digits = len(image['boxes'])
    if no_of_digits > 5:
        more_then_5_counter += 1
        continue

    dig = np.array([])
    dig = np.append(dig, no_of_digits)

    for j in range(no_of_digits) :
        digit = image['boxes'][j]['label']
        dig = np.append(dig, digit)

    zeros = 5 - no_of_digits
    for z in range(zeros) :
        dig = np.append(dig, 0)
        
        
    im = im[:,:,np.newaxis]
    extra_images[i,:,:,:] = im[:,:,:]
    extra_labels[i,:] = dig
    
    if i % 25000 == 0:
        print('progress : ', (i/l)*100, '%')
        
print('Cases Containing More then 5 Digits : ', more_then_5_counter)

In [ ]:
def randomize_dataset(images, labels):
    shuffle = list(zip(images, labels))
    np.random.shuffle(shuffle)
    i, l = zip(*shuffle)
    i, l = np.asarray(i), np.asarray(l)
    return i, l

In [ ]:
%whos

In [ ]:
test_images, test_labels = randomize_dataset(test_images, test_labels)
train_images, train_labels = randomize_dataset(train_images, train_labels)
extra_images, extra_labels = randomize_dataset(extra_images, extra_labels)

In [ ]:
validation_set_size = 5500

In [ ]:
valid_images = extra_images[:validation_set_size]
valid_labels = extra_labels[:validation_set_size]

In [ ]:
left_extra_images = extra_images[:-validation_set_size]
left_extra_labels = extra_labels[:-validation_set_size]

In [ ]:
train_new_images = np.concatenate((left_extra_images,train_images),axis=0)
train_new_labels = np.concatenate((left_extra_labels,train_labels),axis=0)

In [ ]:
hdf_file = 'datasets/pickles/SVHN_multi.hdf5'

hdf = h5py.File(hdf_file, 'w')

with hdf as hf:
    hf.create_dataset("train_images",  data=train_new_images)
    hf.create_dataset("train_labels",  data=train_new_labels)
    hf.create_dataset("valid_images",  data=valid_images)
    hf.create_dataset("valid_labels",  data=valid_labels)
    hf.create_dataset("test_images",  data=test_images)
    hf.create_dataset("test_labels",  data=test_labels)
    
print('SVHN Datasets ready in SVHN_multi.hdf5')